<a href="https://colab.research.google.com/github/minsaee/ai_chat_openAI/blob/master/007_langchain_prompt_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

프롬프트는 LLM에서 최적의 답변을 도출하기 위한 모듈이다.

# 프롬프트 템플릿이란?
- 랭체인의 답변 템플릿은 사용자 입력으로 부터 답변을 생성하기 위한 템플릿이다. 프롬프트 템플릿에는 LLM에 대한 지시, LLM에 대한 질문, LLM이 더 나은 답변을 작성하는데 도움이 되는 답변 예시와 같은 정보가 포함된다.
- 랭체인에서 제공하는 프롬프트 템플릿 목록
  - PromptTemplate :  프롬프트 템플릿 기본 클래스
  - FewShotPromptTemplate : 답변 예시가 있는 프롬프트 템플릿
  

# 랭체인 사전 준비

In [ ]:

# 패키지 설치
!pip install langchain==0.0.354
!pip install openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# 환경변수 준비
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
# 패키지 설치
!pip install tiktoken
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00


# 프롬프트 템플릿 생성
```python
PromptTemplate(
    input_variables=[],
    template="멋진 동물이라고 하면?"
)
```
- PromptTempalte() 매개변수

|매개변수|설명|
|---|---|
|input_variables|입력 변수 이름의 배열|
|template|템플릿 문자열|

①  입력 변수가 없는 프롬프트 템플릿
- 입력변수가 없는 경우 PromptTemplate 클래스의 생성자에서 input_variables에 빈 배열을 지정한다.
- 프롬프트 템플릿을 생성한 후 format( )으로 프롬프트를 생성한다.
- 입력변수가 없으면 설정된 템플릿의 문자열이 그대로 출력된다.

In [ ]:
from langchain.prompts import PromptTemplate

# 입력 변수가 없는 프롬프트 템플릿 만들기
no_input_prompt = PromptTemplate(
    input_variables=[],
    template="멋진 동물이라고 하면?"
)

# 프롬프트 생성
print(no_input_prompt.format())

멋진 동물이라고 하면?


② 하나의 입력 변수가 있는 프롬프트 템플릿
- 입력변수가 없는 경우 PromptTemplate 클래스의 생성자에서 input_variables에 입력변수 이름의 배열을 지정하고, format( )에 입력변수 값을 전달하면 템플릿의 '{입력변수명}'에 입력 변수의 값이 삽입된다.


In [ ]:
from langchain.prompts import PromptTemplate

# 하나의 입력 변수가 있는 프롬프트 템플릿 만들기
one_input_prompt = PromptTemplate(
    input_variables=["content"],
    template="멋진 {content}이라고 하면?"
)

# 프롬프트 생성
print(one_input_prompt.format(content="동물"))

멋진 동물이라고 하면?


③ 여러 입력 변수가 있는 프롬프트 템플릿

In [ ]:
from langchain.prompts import PromptTemplate

# 여러 개의 입력 변수가 있는 프롬프트 템플릿 만들기
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="{adjective}{content}이라고 하면?"
)

# 프롬프트 생성
print(multiple_input_prompt.format(adjective="멋진", content="동물"))

멋진동물이라고 하면?


④ Jinja2로 프롬프트 템플릿 만들기
- 랭체인의 프롬프트 템플릿은 템플릿 엔진의 Jinja2의 서식도 지원한다.
- Jinja2는 파이썬의 템플릿 엔진 중 하나로, 동적인 웹 페이지를 만드는데 사용된다.
- Jinja공식문서 https://jinja.palletsprojects.com/en/3.1.x/

In [ ]:
from langchain.prompts import PromptTemplate

# jinja2를 이용한 프롬프트 템플릿 준비
jinja2_prompt = PromptTemplate(
    input_variables=["items"],
    template_format="jinja2",
    template="""
{% for item in items %}
Q: {{ item.question }}
A: {{ item.answer }}
{% endfor %}
"""
)

# 프롬프트 생성
items=[
    {"question": "foo", "answer": "bar"},
    {"question": "1", "answer": "2"}
]
print(jinja2_prompt.format(items=items))



Q: foo
A: bar

Q: 1
A: 2



# 답변 예시가 포함된 프롬프트 템플릿
- 답변 예시가 포함된 프롬프트 템플릿을 만들려면 FewShotPromptTemplate 클래스를 사용한다.

- FewShotPromptTemplate(  ) 주요 매개변수

|매개변수|설명|
|---|---|
|examples|답변예시|
|example_prompt|프롬프트 템플릿|
|prefix|접두사|
|suffix|접미사|
|input_variables|입력변수|
|example_separator|구분기호|

- 프롬프트에 답변 예시를 포함시킴으로써 LLM 답변의 정확도를 높일 수 있다.


In [ ]:
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# 답변 예시를 포함한 프롬프트 템플릿 만들기
prompt_from_string_examples = FewShotPromptTemplate(
    examples=examples, # 답변 예시
    example_prompt=example_prompt, # 프롬프트 템플릿
    prefix="모든 입력에 대한 반의어를 입력하세요", # 접두사
    suffix="입력: {adjective}\n출력:", # 접미사
    input_variables=["adjective"], # 입력 변수
    example_separator="\n\n" # 구분 기호

)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 밝은
출력: 어두운

입력: 재미있는
출력: 지루한

입력: 큰
출력:


# 다양한 답변 예시가 포함된 프롬프트 템플릿
- 여러개의 답변 예시가 포함된 프롬프트 템플릿을 만들려면 ExampleSelector 클래스를 상속받아 사용한다.
- 답변 예시가 여러 개인 경우 어떤 예시를 사용할지 선택하는 선택기가 된다.
   - LengthBasedExampleSelector
   - SemanticSimilarityExampleSelector
   - MaxMarginalRelevanceExampleSelector

# LengthBasedExampleSelector
- LengthBasedExampleSelector는 문자열 길이를 기준으로 사용할 답변 예시를 선택한다.
- 이것은 입력된 토큰 수가 최대 토큰 수를 초과할 우려가 있을 때 유용하다.
- 입력이 길면 답변예시를 적게 포함하고, 입력이 짧으면 답변 예시를 많이 포함한다.
- LengthBasedExampleSelector( ) 주요 매개변수

|매개변수|설명|
|---|---|
|examples|답변예시|
|example_prompt|프롬프트 템플릿|
|max_length|문자열의 최대 길이|


- 사용할 답변 선택기는 FewShotPrompTemplate 클래스의 example_selector에 지정한다.

In [ ]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# LengthBasedExampleSelector 생성
example_selector = LengthBasedExampleSelector(
    examples=examples, # 답변 예시
    example_prompt=example_prompt, # 프롬프트 템플릿
    max_length=10, # 문자열의 최대 길이
)

# FewShotPromptTemplate 생성
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"
)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 밝은
출력: 어두운

입력: 재미있는
출력: 지루한

입력: 큰
출력:


# SemanticSimilarityExampleSelector
- SemanticSimilarityExampleSelector는 입력과 가장 유사한 답변 예제를 기준으로 답변 예제를 선택한다.
- 입력과 코사인 유사도(consine similarity)가 가장 높은 임베딩을 사용해 답변 예시를 찾는다.
- SemanticSimilarityExampleSelector.from_examples( ) 주요 매개변수

|매개변수|설명|
|---|---|
|examples|답변예시|
|embeddings|임베디드 생성 클래스|
|vectorstore_cls|임베디드 유사 검색 클래스|
|k|답변 예시 개수|

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# SemanticSimilarityExampleSelector 생성
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, # 답변 예시
    embeddings=OpenAIEmbeddings(), # 임베디드 생성 클래스
    vectorstore_cls=FAISS, # 임베디드 유사 검색 클래스
    k=3 # 답변 예시 개수
)

# FewShotPromptTemplate 생성
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"
)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))

모든 입력에 대한 반의어를 입력하세요

입력: 밝은
출력: 어두운

입력: 재미있는
출력: 지루한

입력: 높은
출력: 낮은

입력: 
출력:


# MaxMarginalRelevanceExampleSelector
- MaxMarginalRelevanceExampleSelector는 다양성을 최적화하면서 입력과 가장 유사한 답변 예시 조합을 기반으로 답변 예시를 선택한다.
- 입력과 코사인 유사도가 가장 높은 답변 예시를 찾으면서 이미 선택된 답변 예시와 유사한 답변 예시는 포함하지 않도록 한다.

- MaxMarginalRelevanceExampleSelector.from_examples( )주요 매개변수

|매개변수|설명|
|---|---|
|examples|답변 예시|
|embeddings|임베디드 생성 클래스|
|vectorstore_cls|임베디드 유사 검색 클래스|
|k|답변 예시 개수|


In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 답변 예시 준비
examples = [
    {"input": "밝은", "output": "어두운"},
    {"input": "재미있는", "output": "지루한"},
    {"input": "활기찬", "output": "무기력한"},
    {"input": "높은", "output": "낮은"},
    {"input": "빠른", "output": "느린"},
]

# 프롬프트 템플릿 생성
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="입력: {input}\n출력: {output}",
)

# MaxMarginalRelevanceExampleSelector 생성
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples, # 답변 예시
    embeddings=OpenAIEmbeddings(), # 임베디드 생성 클래스
    vectorstore_cls=FAISS, # 임베디드 유사 검색 클래스
    k=3 # 답변 예시 개수
)

# FewShotPromptTemplate 준비
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="모든 입력에 대한 반의어를 입력하세요",
    suffix="입력: {adjective}\n출력:",
    input_variables=["adjective"],
    example_separator="\n\n"

)

# 프롬프트 생성
print(prompt_from_string_examples.format(adjective="큰"))